In [40]:
import glob
import os
import pandas as pd
print("Priyatama is ready!")

Priyatama is ready!


## Merge multiple CSV into one big data frame.

In [41]:
# Provide path to folder
path = r"C:\gabschool\The Analysts' Upgrade\Data & PreProcessing\raw_data"

# List all CSV files, excluding those with '_info' in the filename
all_files = glob.glob(os.path.join(path, '*.csv'))
filtered_files = [f for f in all_files if '_info' not in os.path.basename(f)]

# Create a list to store individual DataFrames
df_list = []

# Loop through filtered files to read and append them
for file in filtered_files:
    df = pd.read_csv(file)
    match_id = os.path.splitext(os.path.basename(file))[0]
    df['match_id'] = match_id
    df_list.append(df)

# Combine all DataFrames into one
combined_df = pd.concat(df_list, ignore_index=True)

# Display information about the combined DataFrame
combined_df.shape

(276087, 22)

In [42]:
df = combined_df.copy()
df['start_date'] = pd.to_datetime(df['start_date'])
# Create a new column 'year' containing the year extracted from 'start_date'
df['year'] = df['start_date'].dt.year

In [43]:
# Mapping of full team names to initials
team_name_map = {
    "Mumbai Indians": "MI",
    "Chennai Super Kings": "CSK",
    "Royal Challengers Bengaluru": "RCB",
    "Kolkata Knight Riders": "KKR",
    "Sunrisers Hyderabad": "SRH",
    "Delhi Capitals": "DC",
    "Punjab Kings": "PBKS",
    "Rajasthan Royals": "RR",
    "Lucknow Super Giants": "LSG",
    "Gujarat Titans": "GT"
}

# Replace team names in batting_team and bowling_team
df["batting_team"] = df["batting_team"].replace(team_name_map)
df["bowling_team"] = df["bowling_team"].replace(team_name_map)

In [44]:
df['start_date'] = pd.to_datetime(df['start_date'])
df = df[df['start_date'].dt.year == 2025]

In [45]:

# Step 1: Compute total runs per delivery
df['total_runs'] = df['runs_off_bat'] + df['extras']

# Step 2: Aggregate total runs per match and batting team
match_team_runs = (
    df.groupby(['match_id', 'batting_team'])['total_runs']
    .sum()
    .reset_index()
)

# Step 3: Determine the winning team for each match
# This will get the team with the max total_runs per match
winning_teams = match_team_runs.loc[
    match_team_runs.groupby('match_id')['total_runs'].idxmax()
].copy()

# Rename for merging
winning_teams = winning_teams[['match_id', 'batting_team']].rename(
    columns={'batting_team': 'winning_team'}
)

# Step 4: Merge the winning team info back into the original DataFrame
df = df.merge(winning_teams, on='match_id', how='left')


In [46]:
df.to_csv('ipl_2025.csv',index=False)

In [47]:
df.head()

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,year,total_runs,winning_team
0,1473438,2025,2025-03-22,"Eden Gardens, Kolkata",1,0.1,KKR,RCB,Q de Kock,SP Narine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,0,RCB
1,1473438,2025,2025-03-22,"Eden Gardens, Kolkata",1,0.2,KKR,RCB,Q de Kock,SP Narine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,4,RCB
2,1473438,2025,2025-03-22,"Eden Gardens, Kolkata",1,0.3,KKR,RCB,Q de Kock,SP Narine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,0,RCB
3,1473438,2025,2025-03-22,"Eden Gardens, Kolkata",1,0.4,KKR,RCB,Q de Kock,SP Narine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,0,RCB
4,1473438,2025,2025-03-22,"Eden Gardens, Kolkata",1,0.5,KKR,RCB,Q de Kock,SP Narine,...,NaN,NaN,NaN,caught,Q de Kock,NaN,NaN,2025,0,RCB
